<a href="https://colab.research.google.com/github/jchen42703/MathResearchQHSS/blob/active-dev/lipreading/Lipreading_Training_Demo_[Cleaner].ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1710/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1710/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package lists... Done


In [0]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.4-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [0]:
! pip install ffmpeg sk-video

In [0]:
# if the top two don't work, run: 
!apt-get install --no-install-recommends ffmpeg && pip install ffmpeg scikit-video

# IO
* Uploading data through google drive; ask jchen42703@gmail.com or j.chen3@qhss.org to share it with you. 

In [0]:
import os
!mkdir s1
!mkdir s1_align
os.listdir()

!pip install -U -q PyDrive

s1_folder_id = '1B8cIYz6ljEbjYapG6EW1NrKHgcpZ2HzD'
s1_align_folder_id = '1swcuyFY-ZdNgFBou5GiigfEXsURcPO63'

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

mkdir: cannot create directory ‘s1’: File exists
mkdir: cannot create directory ‘s1_align’: File exists


In [0]:
# uploading all of speaker 1's .mpg files.
%cd '/content/s1'
!pwd

file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(s1_folder_id)}).GetList()

for file1 in sorted(file_list, key = lambda x: x['title']):
    print('Downloading {} from GDrive'.format(file1['title'])) # i+1, len(file_list)))
    file1.GetContentFile(file1['title'])

In [0]:
# uploading all of the corresponding align files (labels)
cd '/content/s1_align'
!pwd

file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(s1_align_folder_id)}).GetList()

for file1 in sorted(file_list, key = lambda x: x['title']):
    print('Downloading {} from GDrive'.format(file1['title'])) # i+1, len(file_list)))
    file1.GetContentFile(file1['title'])

In [0]:
# check directories
%cd '/content/'
# os.listdir('/content'), os.listdir('/content/training'), os.listdir('/content/labels')

/content


## Model + IO


In [0]:
! rm -r MathResearchQHSS
! git clone -b lipreading https://github.com/jchen42703/MathResearchQHSS.git

Cloning into 'MathResearchQHSS'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 362 (delta 39), reused 0 (delta 0), pack-reused 306
Receiving objects: 100% (362/362), 21.20 MiB | 8.43 MiB/s, done.
Resolving deltas: 100% (187/187), done.


In [0]:
%cd MathResearchQHSS
!pip install -r requirements.txt
%cd '/content/'

/content/MathResearchQHSS
/content


In [0]:
from MathResearchQHSS.lipreading.io.generator import FrameGenerator
from MathResearchQHSS.lipreading.models import LipNet
from MathResearchQHSS.lipreading.io.io_utils import get_list_IDs
import os
from glob import glob
import skvideo

old = (75, 576, 720, 3)
new = (75, 192, 240,3)

s1_path = '/content/s1/'
s1_align_path = '/content/s1_align/'

# initializing generators 
list_IDs = get_list_IDs(s1_path, val_split = 0.8)
data_dirs = [s1_path, s1_align_path]
train_gen = FrameGenerator(list_IDs['train'], data_dirs, batch_size = 1, resize_shape = new )
val_gen = FrameGenerator(list_IDs['val'], data_dirs, batch_size = 1, resize_shape = new)
# gen.__getitem__(1)

In [0]:
# quick testing of generators
inp, out = train_gen.__getitem__(1)
inp['the_input'].shape

/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


(1, 75, 192, 240, 3)

In [0]:
# initializing model (takes a while)
lipnet = LipNet(img_w = new[1], img_h = new[2])
lipnet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
the_input (InputLayer)       (None, 75, 192, 240, 3)   0         
_________________________________________________________________
zero1 (ZeroPadding3D)        (None, 77, 196, 244, 3)   0         
_________________________________________________________________
conv1 (Conv3D)               (None, 75, 96, 120, 32)   7232      
_________________________________________________________________
max1 (MaxPooling3D)          (None, 75, 48, 60, 32)    0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 75, 48, 60, 32)    0         
_________________________________________________________________
zero2 (ZeroPadding3D)        (None, 77, 52, 64, 32)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 75, 48, 60, 64)    153664    
__________

In [0]:
# compiles model
from keras.optimizers import Adam
adam = Adam(lr=3e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

# the loss calc occurs elsewhere, so use a dummy lambda func for the loss
lipnet.model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer=adam)

In [0]:
# training 
n_epochs = 20
max_queue_size = 1
lipnet.model.fit_generator(generator = train_gen, epochs = n_epochs, max_queue_size = max_queue_size, workers = 2, use_multiprocessing = True)

Epoch 1/200


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


390/390 [==============================] - 1173s 3s/step - loss: 63.6387
Epoch 2/200


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


  8/390 [..............................] - ETA: 21:08 - loss: 59.2284

Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py", line 401, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/content/MathResearchQHSS/lipreading/io/io_utils.py", line 75, in __getitem__
    X, y = self.data_gen(list_IDs_temp)
  File "/content/MathResearchQHSS/lipreading/io/generator.py", line 52, in da

KeyboardInterrupt: ignored

In [0]:
model.save_weights('train_weights_'+str(n_epochs)+'.h5')

In [0]:
# Adding CLR
! git clone https://github.com/jchen42703/CLR.git

Cloning into 'CLR'...
remote: Enumerating objects: 247, done.
remote: Total 247 (delta 0), reused 0 (delta 0), pack-reused 247
Receiving objects: 100% (247/247), 1.37 MiB | 1.29 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [0]:
# initializing the cyclical learning rate callback
from CLR import clr_callback
clr = clr_callback.CyclicLR(base_lr=1e-5, max_lr=3e-3, 
                        step_size=1000., # originally 1000
                        mode = 'triangular')
                        #mode='exp_range', gamma=0.99994)
callbacks = [clr]

In [0]:
# training 
n_epochs = 20
max_queue_size = 1
lipnet.model.fit_generator(generator = train_gen, epochs = n_epochs, callbacks = callbacks, max_queue_size = max_queue_size, workers = 2, use_multiprocessing = True)

Epoch 1/200


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


  5/390 [..............................] - ETA: 23:21 - loss: 56.6298

Process ForkPoolWorker-9:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process ForkPoolWorker-10:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py", line 401, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/content/MathResearchQHSS/lipreading/io/io_utils.py", line 75, in __getitem__
    X, y = self.data_gen(list_IDs_temp)
  File "/content/MathResearchQHSS/lipreading/io/generator.py", line 52, in data_gen
    load_data = np.asarray(skvideo.io.vread(file_x))
  File "/usr/local/lib/python3.6/dist-packages/skvideo/io/io.py", line 144, in vread
    reader = FFmpegReader(fname, inputdict=inputdict, outputdict=outputdict, verbosity=verbosity)
  Fi

Epoch 1/200


  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py", line 401, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/content/MathResearchQHSS/lipreading/io/io_utils.py", line 75, in __getitem__
    X, y = self.data_gen(list_IDs_temp)
  File "/content/MathResearchQHSS/lipreading/io/generator.py", line 54, in data_gen
    video = np.asarray([resize(load_data[i], self.resize_shape[-3:]) for i in range(old[0])])
  File "/content/MathResearchQHSS/lipreading/io/generator.py", line 54, in <listcomp>
    video = np.asarray([resize(load_data[i], self.resize_shape[-3:]) for i in range(old[0])])
  File "/usr/local/lib/python3.6/dist-packages/skimage

KeyboardInterrupt: ignored